In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AddedToken
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import LocalDataset
import numpy as np
from tqdm import tqdm
from multiprocess import Pool
import itertools
import json
import os

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

class UInt32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint32)

_encodings['uint32'] = UInt32

columns = {
    'input_ids': 'uint32',
    'position_ids': 'uint32',
    'attention_mask': 'uint32',
    'audio': 'str',
    'text': 'str'
}
hashes = 'sha1', 'xxh64'

datasets = ['ayat_aktif_pasif', 'coding', 'malaysian_reasoning', 'meta_prompt', 'multiple_choice_qa']

/root/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-32B')

In [3]:
ds = load_dataset("Scicom-intl/Malaysian-Instructions", datasets[0])

In [4]:
data = []

ds = load_dataset("Scicom-intl/Malaysian-Instructions", 'ayat_aktif_pasif')
for i in range(len(ds['train'])):
    messages = [
        {'role': 'user', 'content': ds['train'][i]['input']},
        {'role': 'assistant', 'content': ds['train'][i]['output']}
    ]
    data.append(messages)

In [5]:
ds = load_dataset("Scicom-intl/Malaysian-Instructions", 'coding')
for i in range(len(ds['train'])):
    messages = [
        {'role': 'user', 'content': ds['train'][i]['question']},
        {'role': 'assistant', 'content': ds['train'][i]['answer']}
    ]
    data.append(messages)

In [6]:
ds = load_dataset("Scicom-intl/Malaysian-Instructions", 'malaysian_reasoning')
for i in range(len(ds['train'])):
    messages = [
        {'role': 'system', 'content': ds['train'][i]['system']},
        {'role': 'user', 'content': ds['train'][i]['input']},
        {'role': 'assistant', 'content': ds['train'][i]['answer'], 'reasoning_content': ds['train'][i]['reasoning']}
    ]
    data.append(messages)

In [ ]:
ds = load_dataset("Scicom-intl/Malaysian-Instructions", 'meta_prompt')
for i in range(len(ds['train'])):
    messages = [
        {'role': 'user', 'content': ds['train'][i]['input']},
        {'role': 'assistant', 'content': ds['train'][i]['answer']}
    ]
    data.append(messages)

In [ ]:
ds = load_dataset("Scicom-intl/Malaysian-Instructions", 'multiple_choice_qa')
for i in range(len(ds['train'])):
    messages = [
        {'role': 'user', 'content': ds['train'][i]['question']},
        {'role': 'assistant', 'content': ds['train'][i]['answer']}
    ]
    data.append(messages)

In [ ]:
len(data)

In [ ]:
!rm -rf tokenized-qwen3

In [ ]:
def collator(batch, batch_position_ids):
    input_ids = []
    position_ids = []
    masks = []
    for i in range(len(batch)):
        l = len(batch[i])
        input_ids.extend(batch[i])
        position_ids.extend(batch_position_ids[i])
        masks.append(l)
    
    return {
        'input_ids': np.array(input_ids).astype(np.uint32),
        'position_ids': np.array(position_ids).astype(np.uint32),
        'attention_mask': np.array(masks).astype(np.uint32),
        'audio': '',
        'text': '',
    }

sequence_length = 1024 * 16
def loop(files, block_size = sequence_length):
    rows, index = files
    out_root = f'tokenized-qwen3/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    count = 0
    temp = []
    position_ids = []
    last_block, last_position_block = None, None
    with MDSWriter(out=out_root, columns=columns, compression=None, hashes=hashes) as out:
        for row in tqdm(rows):

            t = tokenizer.apply_chat_template(row, tokenize=False)
            outputs = tokenizer(t, add_special_tokens=False)
            position = range(len(outputs['input_ids']))
            length = len(outputs['input_ids'])

            if length > block_size:
                continue
            
            if count + length > block_size:
                o = collator(temp, position_ids)
                if o['input_ids'].shape[0] > 0:
                    out.write(o)
                temp = [outputs['input_ids']]
                position_ids = [position]
                count = length
                
            else:
                temp.append(outputs['input_ids'])
                position_ids.append(range(len(outputs['input_ids'])))
                count += len(outputs['input_ids'])
        
        if len(temp):
            o = collator(temp, position_ids)
            if o['input_ids'].shape[0] > 0:
                out.write(o)

In [ ]:
loop((data[:100], 0))

In [ ]:
multiprocessing(data, loop, cores = 20, returned=False)

In [ ]:
from glob import glob

folders = sorted(glob('tokenized-qwen3/tokenized-*'), key = lambda x: int(x.split('-')[-1]))
folders

In [ ]:
!rm -rf multipacking-qwen3

In [ ]:
with MDSWriter(out='multipacking-qwen3', columns=columns, compression=None, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = LocalDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

In [ ]:
dataset = LocalDataset('multipacking-qwen3')
len(dataset)

In [ ]:
dataset[0]